#Applied Data Science Capstone

##Exploring Business Opportunity for a Grocery Store location in Richmond, Virginia

###Load Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")


     |████████████████████████████████| 102kB 10.9MB/s ta 0:00:01
Libraries imported.


###Get Data from source into dataframe

In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Richmond,_Virginia").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
# create a new DataFrame from the list
rva_df = pd.DataFrame({"Neighborhood": neighborhoodList})

rva_df.head(10)

,Neighborhood
0,"List of neighborhoods in Richmond, Virginia"
1,"Neighborhoods of Richmond, Virginia"
2,"Arts District, Richmond, Virginia"
3,Barton Heights
4,Biotech and MCV District
5,"Brandermill, Virginia"
6,"Carillon, Richmond, Virginia"
7,"Carver, Richmond, Virginia"
8,"Carytown, Richmond, Virginia"
9,Central Office District


In [7]:
rva_df.shape

(65, 1)

###Drop non-neighborhood rows

In [8]:
rva_df.drop([0,1], axis=0).head(10)

,Neighborhood
2,"Arts District, Richmond, Virginia"
3,Barton Heights
4,Biotech and MCV District
5,"Brandermill, Virginia"
6,"Carillon, Richmond, Virginia"
7,"Carver, Richmond, Virginia"
8,"Carytown, Richmond, Virginia"
9,Central Office District
10,Chamberlayne Industrial Center
11,Chestnut Hill–Plateau Historic District


###Get coordinates for the neighborhoods

In [9]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Richmond,Virginia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
coords = [ get_latlng(neighborhood) for neighborhood in rva_df["Neighborhood"].tolist() ]

In [11]:
coords

[[37.565800000000024, -77.44863999999995],
 [37.565800000000024, -77.44863999999995],
 [37.54158008050089, -77.43499994503438],
 [37.56891519938178, -77.43176533906961],
 [37.540680000000066, -77.43366999999995],
 [37.43209000000007, -77.64970999999997],
 [37.543660000000045, -77.48443999999995],
 [37.55323000000004, -77.44939999999997],
 [37.553270000000055, -77.48231999999996],
 [37.534194, -77.51904524999999],
 [37.62724000000003, -77.43135999999998],
 [37.562400000000025, -77.42180999999994],
 [37.531510000000026, -77.41748999999999],
 [37.555600500000004, -77.42597849999999],
 [37.56331560909512, -77.36790282289435],
 [37.54432000000003, -77.44459999999998],
 [37.534026865174454, -77.43331515189351],
 [37.541540000000055, -77.41363999999999],
 [37.54737000000006, -77.46414999999996],
 [37.522740000000056, -77.48059999999998],
 [37.51838883976256, -77.4085095],
 [37.53826000000004, -77.44684999999998],
 [37.534026865174454, -77.43331515189351],
 [37.582340000000045, -77.45070999999

In [12]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [13]:
# merge the coordinates into the original dataframe
rva_df['Latitude'] = df_coords['Latitude']
rva_df['Longitude'] = df_coords['Longitude']

In [14]:
# check the neighborhoods and the coordinates
print(rva_df.shape)
rva_df

(65, 3)


,Neighborhood,Latitude,Longitude
0,"List of neighborhoods in Richmond, Virginia",37.565800,-77.448640
1,"Neighborhoods of Richmond, Virginia",37.565800,-77.448640
2,"Arts District, Richmond, Virginia",37.541580,-77.435000
3,Barton Heights,37.568915,-77.431765
4,Biotech and MCV District,37.540680,-77.433670
5,"Brandermill, Virginia",37.432090,-77.649710
6,"Carillon, Richmond, Virginia",37.543660,-77.484440
7,"Carver, Richmond, Virginia",37.553230,-77.449400
8,"Carytown, Richmond, Virginia",37.553270,-77.482320
9,Central Office District,37.534194,-77.519045


In [15]:
# save the DataFrame as CSV file
rva_df.to_csv("rva_df.csv", index=False)

###Create a map of Richmond, Virginia neighborhoods

In [16]:
# get the coordinates of Richmond, Virginia
address = 'Richmond, Virginia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Richmond, Virginia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Richmond, Virginia 37.5385087, -77.43428.


In [17]:
# create map of Richmond, Virginia using latitude and longitude values
map_rva = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(rva_df['Latitude'], rva_df['Longitude'], rva_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_rva)  
    
map_rva

In [18]:
# save the map as HTML file
map_rva.save('map_rva.html')

###Foursquare API Data for Richmond, Virginia Neighborhoods

In [19]:
# define Foursquare Credentials and Version
CLIENT_ID = 'DBSBTIR0LVBY4Q4KIRSMU1FVNTJ5RANYXUVBAEIUNIWKZ3NQ' # your Foursquare ID
CLIENT_SECRET = 'BX3TX53D0MXIXELMMMHWIQVAQV5WBU1IZCLVTJJY54VZ4PSY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DBSBTIR0LVBY4Q4KIRSMU1FVNTJ5RANYXUVBAEIUNIWKZ3NQ
CLIENT_SECRET:BX3TX53D0MXIXELMMMHWIQVAQV5WBU1IZCLVTJJY54VZ4PSY


###Use Foursquare data to look at the venues in the neighborhoods

In [22]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(rva_df['Latitude'], rva_df['Longitude'], rva_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [23]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4765, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"List of neighborhoods in Richmond, Virginia",37.5658,-77.44864,SCOR: Sports Center of Richmond,37.563313,-77.455388,Gym
1,"List of neighborhoods in Richmond, Virginia",37.5658,-77.44864,Hardywood Park Craft Brewery,37.564810,-77.457135,Brewery
2,"List of neighborhoods in Richmond, Virginia",37.5658,-77.44864,Sugar Shack Donuts,37.557084,-77.453144,Donut Shop
3,"List of neighborhoods in Richmond, Virginia",37.5658,-77.44864,Food Truck Court At Hardywood,37.564559,-77.457302,Food Court
4,"List of neighborhoods in Richmond, Virginia",37.5658,-77.44864,CrossFit RVA,37.567474,-77.463065,Gym


###Venues by Neighborhood

In [24]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Arts District, Richmond, Virginia",100,100,100,100,100,100
Barton Heights,29,29,29,29,29,29
Biotech and MCV District,100,100,100,100,100,100
"Brandermill, Virginia",21,21,21,21,21,21
"Carillon, Richmond, Virginia",89,89,89,89,89,89
"Carver, Richmond, Virginia",100,100,100,100,100,100
"Carytown, Richmond, Virginia",100,100,100,100,100,100
Central Office District,56,56,56,56,56,56
Chamberlayne Industrial Center,9,9,9,9,9,9


###Categories of Venues

In [25]:
print('There are {} categories in Richmond, Virginia.'.format(len(venues_df['VenueCategory'].unique())))

There are 228 categories in Richmond, Virginia.


In [26]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Gym', 'Brewery', 'Donut Shop', 'Food Court', 'High School',
       'Latin American Restaurant', 'Fried Chicken Joint',
       'American Restaurant', 'Caribbean Restaurant',
       'Thrift / Vintage Store', 'Ice Cream Shop', 'Science Museum',
       'Breakfast Spot', 'Bookstore', 'Burger Joint', 'Cosmetics Shop',
       'Gym / Fitness Center', 'Coffee Shop', 'Seafood Restaurant',
       'Sandwich Place', 'Deli / Bodega', 'Movie Theater', 'Neighborhood',
       'Cuban Restaurant', 'Mediterranean Restaurant', 'Asian Restaurant',
       'Football Stadium', 'Thai Restaurant',
       'Vegetarian / Vegan Restaurant', 'Park', 'Museum', 'BBQ Joint',
       'Mexican Restaurant', 'Music Venue', 'Market', 'Diner',
       'Pizza Place', 'Chinese Restaurant', 'Italian Restaurant',
       'Theater', 'Bakery', 'Modern European Restaurant', 'Food Truck',
       'Beer Store', 'Convenience Store', 'Baseball Stadium',
       'Grocery Store', 'Comfort Food Restaurant', 'College Theater',
       'Ar

In [27]:
# do the results contain "Supermarket"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

###Neighborhood Analysis

In [28]:
# one hot encoding
rva_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
rva_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [rva_onehot.columns[-1]] + list(rva_onehot.columns[:-1])
rva_onehot = rva_onehot[fixed_columns]

print(rva_onehot.shape)
rva_onehot.head()

(4765, 229)


,Neighborhoods,ATM,Adult Boutique,Advertising Agency,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Beach,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recycling Facility,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfall,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"List of neighborhoods in Richmond, Virginia",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"List of neighborhoods in Richmond, Virginia",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

###Grouping by Neighborhood

In [29]:
rva_grouped = rva_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(rva_grouped.shape)
rva_grouped

(65, 229)


,Neighborhoods,ATM,Adult Boutique,Advertising Agency,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Beach,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recycling Facility,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfall,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Arts District, Richmond, Virginia",0.000,0.00,0.010000,0.030000,0.00,0.00,0.040000,0.000000,0.000000,0.010000,0.00000,0.000000,0.010000,0.000000,0.030000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.010000,0.000000,0.00,0.000000,0.010000,0.000000,0.00,0.000000,0.010000,0.00,0.000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.020000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.010000,0.000000,0.020000,0.0000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [30]:
len(rva_grouped[rva_grouped["Supermarket"] > 0])

10

###Richmond, Virginia Supermarket dataframe

In [31]:
rva_sup = rva_grouped[["Neighborhoods","Supermarket"]]

###Cluster Richmond, Virginia Neighborhoods

In [33]:
# set number of clusters
kclusters = 4

rva_clustering = rva_sup.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(rva_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 2], dtype=int32)

In [34]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
rva_merged = rva_sup.copy()

# add clustering labels
rva_merged["Cluster Labels"] = kmeans.labels_

In [35]:
rva_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
rva_merged.head()

,Neighborhood,Supermarket,Cluster Labels
0,"Arts District, Richmond, Virginia",0.0,1
1,Barton Heights,0.0,1
2,Biotech and MCV District,0.0,1
3,"Brandermill, Virginia",0.0,1
4,"Carillon, Richmond, Virginia",0.0,1


In [36]:
# merge Richmond_grouped with Richmond_data to add latitude/longitude for each neighborhood
rva_merged = rva_merged.join(rva_df.set_index("Neighborhood"), on="Neighborhood")

print(rva_merged.shape)
rva_merged.head() # check the last columns!

(65, 5)


,Neighborhood,Supermarket,Cluster Labels,Latitude,Longitude
0,"Arts District, Richmond, Virginia",0.0,1,37.541580,-77.435000
1,Barton Heights,0.0,1,37.568915,-77.431765
2,Biotech and MCV District,0.0,1,37.540680,-77.433670
3,"Brandermill, Virginia",0.0,1,37.432090,-77.649710
4,"Carillon, Richmond, Virginia",0.0,1,37.543660,-77.484440


In [37]:
# sort the results by Cluster Labels
print(rva_merged.shape)
rva_merged.sort_values(["Cluster Labels"], inplace=True)
rva_merged

(65, 5)


,Neighborhood,Supermarket,Cluster Labels,Latitude,Longitude
60,"Washington Park, Richmond",0.023810,0,37.589940,-77.443010
25,"Jahnke, Richmond, Virginia",0.021277,0,37.517070,-77.520420
7,Central Office District,0.017857,0,37.534194,-77.519045
40,"Pine Camp, Richmond, Virginia",0.021277,0,37.602040,-77.448160
44,Shed Town,0.020000,0,37.595884,-77.504261
0,"Arts District, Richmond, Virginia",0.000000,1,37.541580,-77.435000
35,"Neighborhoods of Richmond, Virginia",0.000000,1,37.565800,-77.448640
36,"North Side (Richmond, Virginia)",0.000000,1,37.573980,-77.423276
37,Oakwood–Chimborazo Historic District,0.000000,1,37.528270,-77.407670
38,Old Town Manchester,0.000000,1,37.523340,-77.434810


###Cluster Visualization

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(rva_merged['Latitude'], rva_merged['Longitude'], rva_merged['Neighborhood'], rva_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [39]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

###Clusters 0-3 Analysis

In [40]:
# Cluster 0 Analysis
rva_merged.loc[rva_merged['Cluster Labels'] == 0]

,Neighborhood,Supermarket,Cluster Labels,Latitude,Longitude
60,"Washington Park, Richmond",0.023810,0,37.589940,-77.443010
25,"Jahnke, Richmond, Virginia",0.021277,0,37.517070,-77.520420
7,Central Office District,0.017857,0,37.534194,-77.519045
40,"Pine Camp, Richmond, Virginia",0.021277,0,37.602040,-77.448160
44,Shed Town,0.020000,0,37.595884,-77.504261


In [41]:
# Cluster 1 Analysis
rva_merged.loc[rva_merged['Cluster Labels'] == 1]

,Neighborhood,Supermarket,Cluster Labels,Latitude,Longitude
0,"Arts District, Richmond, Virginia",0.0,1,37.541580,-77.435000
35,"Neighborhoods of Richmond, Virginia",0.0,1,37.565800,-77.448640
36,"North Side (Richmond, Virginia)",0.0,1,37.573980,-77.423276
37,Oakwood–Chimborazo Historic District,0.0,1,37.528270,-77.407670
38,Old Town Manchester,0.0,1,37.523340,-77.434810
39,Oregon Hill,0.0,1,37.537830,-77.451110
41,"Randolph, Richmond, Virginia",0.0,1,37.541000,-77.464070
42,Rocketts Landing,0.0,1,37.516329,-77.415125
45,"Sherwood Park (Richmond, Virginia)",0.0,1,37.570157,-77.453746
46,Shockoe Bottom,0.0,1,37.528470,-77.422570


In [42]:
# Cluster 2 Analysis
rva_merged.loc[rva_merged['Cluster Labels'] == 2]

,Neighborhood,Supermarket,Cluster Labels,Latitude,Longitude
9,Chestnut Hill–Plateau Historic District,0.052632,2,37.562400,-77.421810
49,"South Garden, Richmond, Virginia",0.050000,2,37.489226,-77.490587


In [43]:
# Cluster 3 Analysis
rva_merged.loc[rva_merged['Cluster Labels'] == 3]

,Neighborhood,Supermarket,Cluster Labels,Latitude,Longitude
43,Sauer's Gardens,0.010000,3,37.572710,-77.486450
34,Near West End,0.013889,3,37.645015,-77.542156
29,Malvern Gardens,0.010000,3,37.568730,-77.489510


##Conclusions from Cluster Analysis

####Cluster 2 shows the highest area of supermarkets within the Richmond metro area.  Clusters 0 and 3 show moderate population of supermarkets in the area.  Cluster 1 shows great opportunity for opening a supermarket within that cluster to service those neighborhoods.  It interesting to note that the inner city neighborhoods could be serviced by smaller grocery stores and small markets.  This does present an opportunity for a supermarket chain to enter Richmond, however an assessment on available real estate and proximity to those inner city neighbhorhoods would need to studied for a complete business case.  Based on the Richmond neighborhood data and Foursquare data, an opportunity for a supermarket to service Cluster 1 exists.